# Qcodes example with Keysight 344xxA

`344xxA` models of Keysight digital multimeters have similar QCoDeS drivers. In this tutorial, `Keysight_34465A` is chosen for showcasing the usage of the instrument.

Note however that not every feature/parameter is available on all `344xxA` models. This, when possible, is reflected in the instantiated driver object. Also note that models like `34465A` have options (like `DIG` and `MEM`) that can either be enabled or disabled on a particular instrument; this also has impact on availability of some features/parameters and/or their settings. In general, refer to the instrument's manual for detailed information.

The driver does not cover all the features of the instrument because it is being mostly used for voltage DC measurements. Contribution is welcome.

In [1]:
import time

import qcodes
from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A

In [2]:
dmm = Keysight_34465A('dmm', 'USB0::10893::257::MY57504787::0::INSTR')

Connected to: Keysight Technologies 34465A (serial:MY57504787, firmware:A.02.16-02.40-02.16-00.51-03-01) in 0.48s


## Parameters overview:

Here we present __some__ of the parameters of the driver.

* Single-point measurement
  * `dmm.volt` - measure one voltage point now (note: may not always work)
* Range settings
  * `dmm.range`
  * `dmm.autorange`
* Triggering settings
  * `dmm.trigger_source`
  * `dmm.trigger_delay`
  * `dmm.auto_trigger_delay_enabled`
  * `dmm.trigger_count`
  * `dmm.trigger_slope`
  * `dmm.trigger_level`
* Sample settings
  * `dmm.sample_count`
  * `dmm.sample_source`
  * `dmm.sample_timer`
* Display control
  * `dmm.display_text`
  * `dmm.display_clear`
  * `dmm.display_enabled`
* Related measurement accuracy
  * `dmm.NPLC`
  * `dmm.resolution`
  * `dmm.line_frequency`
  * `dmm.aperture_mode`
  * `dmm.aperture_time`
  * `dmm.autozero`
  
Use `dmm.parameters` (or other ways) to explore all the parameters.

## Single value reading

If one simpy wants to measure a single voltage value right now, the convenient `volt` parameter can be used.

Note that it may not work for some configurations of the instrument, for example, when `sample_count` is not `1`. Later in this notebook, we will present more prowerful and robust ways of performing measurements.

In [3]:
dmm.volt()

0.00927792071

## Multivalue triggered measurements

__NOTE__: Refer to the instrument manual for more information on how to perform measurements with the instrument; here, only the most basic and frequently used ones are demonstated.

Measurements with the instrument are performed in the following way: the instrument's settings are set for a particular kind of measurement, then the measurement is started/initialized, then after all the data has been acquired, it is retrieved from the instrument. Below is an example of such a measurement.

In [4]:
# Use `range`, `autorange` parameters or `autorange_once` method
# to set the measurement range
dmm.autorange_once()

In [5]:
# Todo
# - document measurement process with `read` and `fetch`
#   (consult with experimentalists)
# - add info on how to create array of setpoints 
#   (inst.sample_timer has resolution of some us, 
#    hence it's important to keep that in mind for setpoint values)

In [7]:
# REFERENCE MEASUREMENT SEQUENCE>>>>


dmm.sample_count(3)

# PREPARE

n = dmm.sample_count()

# ensure correct instrument settings

dmm.aperture_mode('OFF')  # aperture mode seems slower ON than OFF
dmm.autorange('OFF')

dmm.trigger_count(1)
dmm.trigger_delay(0)

dmm.sample_count_pretrigger(0)

# Final step

time_per_point = dmm.sample_timer_minimum()

dmm.sample_timer(time_per_point)
dmm.sample_source('TIM')

setpoints = (tuple(np.linspace(0, n*time_per_point, n)),)
shape = (n,)

# MEASURE

n = dmm.sample_count()

# Ensure that the measurement doesn't time out
# TODO (WilliamHPNielsen): What if we wait really long for a trigger?
old_timeout = dmm.visa_handle.timeout
dmm.visa_handle.timeout = old_timeout + n * time_per_point * 1.2 * 1000

# Turn off the display to increase measurement speed
dmm.display_text(f'Acquiring {n} samples')

dmm.init_measurement()

numvals = dmm.fetch()

dmm.visa_handle.timeout = old_timeout

dmm.display_clear()

numvals, setpoints

(array([0.01263095, 0.01261663, 0.01246853]), ((0.0, 6.75e-05, 0.000135),))

## Special values of some parameters

Some parameters can be set to special values like `MIN`/`MAX`/`DEF` which usually mean minimum/maximum/default, respectively. 

In order to obtain the actual value of the parameter that gets set when setting it to one of these special values, just call the get method of the parameter.

In [8]:
# Find out what the maximum value of `sample_timer` can be
dmm.sample_timer('MAX')
dmm.sample_timer()

3600.0

In [9]:
# Find out what the default value of `sample_timer` is
dmm.sample_timer('DEF')
dmm.sample_timer()

1.0

In [10]:
# Find out what the recommended minumum value of `sample_timer` is
dmm.sample_timer('MIN')
dmm.sample_timer()

4.5e-05

In [11]:
# Alternatively, if available, use a conveniently implemented
# get-only parameter to find out the actual value,
# for example, for MIN value of `sample_timer` there is such
# a convenient get-only parameter:
dmm.sample_timer_minimum()

4.5e-05

## Display state impacts command execution speed

Although it is indeed just nice to be able to display useful text on the screen of the instrument, it turn out that disabling the display (or making it display does text instead of all the indicators and values) improves command execution speed from the remote interface and provides basic security (quoted from the instrument's manual).

Hence, the driver provides `display_text` parameter that displays a given text on the insrturment, and a `display_clear` method that clears the text from display.

The driver also provides `display_enabled` parameter. When it is set to `False`, the state of the display is such that it does not show anything. Note, however, that displaying text is still possible when the `display_enabled` is `False` (when `display_enabled` is `False`, `display_clear` clears the text from the screen but does not enable it).

In [12]:
# Displays the text
dmm.display_text('Hello, World!')

In [13]:
# Returns display to its normal state
dmm.display_clear()

In [14]:
# Note that a call to `display_clear` also updates 
# the value of the `display_text` parameter:
assert dmm.display_text() == ''

In [15]:
# Display can also be cleared by setting 
# `display_text` to an empty string
dmm.display_text('some text')  # Displays some text
time.sleep(0.5)
dmm.display_text('')  # Returns display to its normal state

In [16]:
# Disables the display, which makes it turn black
dmm.display_enabled(False)

In [17]:
# Shows some text on a disabled display
dmm.display_text("i'm disabled but still showing text")

In [18]:
# Enabling display in this state 
# won't change what's being displayed
dmm.display_enabled(True)

In [19]:
# ... but since the display is now enabled,
# clearing the display will not only remove the text
# but also show all the normal display indicators.
dmm.display_clear()

## Error handling

Use the following methods to read the error queue of the instrument. The instrument has an error queue of length up to 20 messages. The queue message retrieval is first-in-first-out.

In [21]:
# Retrieve the first (i.e. oldest) error message 
# in the queue (and thereby remove from the queue)
dmm.error()

(0, 'No error')

In [22]:
# The entire queue can be flushed out
# using `flush_error_queue` method.
# Printing the messages in enabled by default
# and can be disables with the `verbose` kwarg.

# generate a few errors
for _ in range(3):
    dmm.write('produce an error!')

In [23]:
dmm.flush_error_queue(verbose=True)

-113 Undefined header
-113 Undefined header
-113 Undefined header
0 No error


## Using data_buffer for triggered multisample measurement (deprecated)

__NOTE__: This approach is deprecated.

One may use `data_buffer` `ArrayParameter` in order to perform a triggered measurement of a predefined number of samples with a single trigger that has not delay. The following code shows how to do it.

__NOTE__: The reason this approach is deprecated is that the internal implementation of `data_buffer` is quite specific and sets up the instrument into a mode that is not generally useful. This is why it is recommended that the users use driver methods like `read` and `fetch` in order to implement the measurements. It is also up to user to decide to wrap his code into his own measurement function/routine or even a QCoDeS parameter for his own convenience.

In [24]:
# Set the trigger mode. For the sake of this example,
# we are going to use "immediate" which will start measuring
# right after `init_measurement` is called (it will be called
# for you by `data_buffer`'s get method')
dmm.trigger_source('IMM')

In [25]:
# Set sample count to number of samples that
# should be acquired
dmm.sample_count(5)

In [26]:
# One needs to prepare the `data_buffer` parameter
# and the instrument for this measurement, otherwise
# getting `data_buffer` will raise an exception
dmm.data_buffer.prepare()

C:\Users\a-miasta\AppData\Local\Continuum\anaconda3\envs\qcodes\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The function "prepare" is deprecated, because the use of this ArrayParameter is limiting as it represents only a single way of performing measurements and performs preparations that are not generally applicable Use "driver methods like `fetch` and `read` (together with other methods and parameters) and custom functions and parameters implemented on top of them" as an alternative.
  after removing the cwd from sys.path.


In [27]:
# Finally perform the measurement,
# and obtain the measured data
# (note the text on the instrument's display
# during the measurement)
data = dmm.data_buffer()

In [28]:
# Now, here's the measured data
data

array([0.01319693, 0.01314679, 0.01309664, 0.01305843, 0.01299873])

In [29]:
# Here are the values of setpoints,
# in this case its time in seconds when
# each of the points have been measured relative
# to the first point
dmm.data_buffer.setpoints

((0.0,
  5.6250000000000005e-05,
  0.00011250000000000001,
  0.00016875,
  0.00022500000000000002),)

In [30]:
# The shape of the setpoints reflects
# the number of samples
dmm.data_buffer.shape

(5,)

In [31]:
# For internal use, the amount of time used for
# measuring every point is saved into the following 
# attribute:
dmm.data_buffer.time_per_point

4.5e-05

In [32]:
# ... which is equal to the value of the
# `sample_timer` parameter:
dmm.sample_timer()

4.5e-05